In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [11]:
R_10 = torch.load("../../cluster_results/results/0428152740_nclusters_10_utahnetwork/best_flows.pt")
R_10 = R_10["LinkFlows"]

/tmp/ipykernel_312355/3583505112.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  R_10 = torch.load("../../cluster_results/results/0428152740_nclusters_10_utahnetwork/bes

In [12]:
print("10", "shape: ", R_10.shape, "sum: ", R_10.sum().item(), "max: ", R_10.max().item(), "min: ", R_10.min().item())

10 shape:  torch.Size([3785, 24]) sum:  51524212.0 max:  6808.24609375 min:  0.0
